### SQL Zillow Project
GitHub: @guzmanwolfrank

Volatile changes are occurring in the United States housing market.  Here we look at the data on Home Value to analyze which locations have the highest market values and which locations have sustained an above average move upward in price.   

In [31]:
#Imports
import sqlite3
import pandas as pd    
import numpy as np 
import seaborn as sns
import matplotlib as plt   
import warnings 
import csv 


# Ignore all warnings 
warnings.filterwarnings('ignore')


# Import CSV file and Convert to a Pandas DataFrame
csv_file = r"C:\Users\Wolfrank\Desktop\Zillow.csv"
df = pd.read_csv(csv_file).round(2)
#display(df)

In [32]:
# Show all columns in df
column_names = df.columns
print(column_names)


Index(['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName',
       '2000-01-31', '2000-02-29', '2000-03-31', '2000-04-30', '2000-05-31',
       ...
       '2022-10-31', '2022-11-30', '2022-12-31', '2023-01-31', '2023-02-28',
       '2023-03-31', '2023-04-30', '2023-05-31', '2023-06-30', '2023-07-31'],
      dtype='object', length=288)


In [ ]:
#Clean Data 

